# APIs: requêtes HTTP

## Imports

In [6]:
import json
import requests
import pandas as pd

print("import effectué")

import effectué


Interrogation d'une API

In [11]:
# Elle permet d'obtenir le résumé d'un article Wikipédia
# Elle peut servir à enrichir le corpus CAMille avec des données contextuelles
# (par exemple sur un journaliste ou un journal belge ancien).

# Exemple : on recherche un résumé sur "Le Soir (journal belge)"
query = "Le_Soir"
url = f"https://fr.wikipedia.org/api/rest_v1/page/summary/{query}"

headers = {
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) '
                  'AppleWebKit/537.36 (KHTML, like Gecko) '
                  'Chrome/39.0.2171.95 Safari/537.36'}


response = requests.get(url, headers=headers)

if response.status_code == 200:
    print(f"Requête réussie pour : {query}")
    data = response.json()
    print("Clés disponibles :", list(data.keys()))
else:
    print(f"Erreur HTTP {response.status_code}")


Requête réussie pour : Le_Soir
Clés disponibles : ['type', 'title', 'displaytitle', 'namespace', 'wikibase_item', 'titles', 'pageid', 'lang', 'dir', 'revision', 'tid', 'timestamp', 'description', 'description_source', 'content_urls', 'extract', 'extract_html']


In [12]:
list(data.keys())

['type',
 'title',
 'displaytitle',
 'namespace',
 'wikibase_item',
 'titles',
 'pageid',
 'lang',
 'dir',
 'revision',
 'tid',
 'timestamp',
 'description',
 'description_source',
 'content_urls',
 'extract',
 'extract_html']

Extration et affichage des champs pertinents

In [13]:
# Extraction des informations pertinentes
# On récupère proprement : titre, description et résumé (extract).

titre = data.get("title", "N/A") #data.get pour eviter les erreurs si des champs sont manquants
description = data.get("description", "N/A")
resume = data.get("extract", "N/A")

# Affichage formaté
print("Titre       :", titre)
print("Description :", description)
print("\nRésumé :\n", resume)

Titre       : Le Soir
Description : quotidien généraliste belge francophone

Résumé :
 Le Soir est un quotidien généraliste belge de langue française fondé en 1887 par Émile Rossel. Il fait partie du Groupe Rossel. Se présentant comme progressiste et indépendant, ce journal est de tradition libérale et historiquement au centre de l'échiquier politique belge.


Mise en tableau

In [14]:
# Mise en tableau des résultats
# On stocke les informations extraites dans un petit DataFrame pandas

api_results = pd.DataFrame([{
    "Sujet": titre,
    "Description": description,
    "Résumé": resume
}])

In [15]:
# On affiche le tableau
api_results

,Sujet,Description,Résumé
0,Le Soir,quotidien généraliste belge francophone,Le Soir est un quotidien généraliste belge de ...


## Utiliser [Nominatim](https://nominatim.org/) pour connaître les coordonnées géographiques d'une adresse

In [2]:
address = "Avenue Franklin Roosevelt, Ixelles"

"""Retrieve coordinates from Open Street Map"""
headers = {'Referer': 'https://nominatim.openstreetmap.org/ui/search.html'}
url = "https://nominatim.openstreetmap.org/search"
data = {'q': address, 'format': 'json'}
resp = requests.get(url, data, headers=headers)
try:
    json_list = json.loads(resp.text)
except json.JSONDecodeError: # response is not in JSON as expected, probably the request is being blocked
    print(resp.text)
for item in json_list:
    display_name = item['display_name']
    short_name = display_name.split(", ")[0]
    lat = item['lat']
    lon = item['lon']
    print(f"{short_name} ({lat} - {lon})")


ULB (50.8116064 - 4.3805110)


## Utiliser REST Countries pour récupérer des informations sur un pays

https://restcountries.com/

In [3]:
country_name = "Cameroon"

base_url = "https://restcountries.com/v3.1/"
name_url = base_url + "name/"
code_url = base_url + "alpha/"
resp = requests.get(name_url + country_name)
country = resp.json()[0]
try:
    languages = country['languages']
    print(f"Languages: {', '.join(languages.values())}")
    border_codes = country['borders']
    border_names = []
    for code in border_codes:
        resp = requests.get(code_url + code)
        border_country = resp.json()[0]
        border_name = border_country["name"]["common"]
        border_names.append(border_name)
    print(f"Borders: {', '.join(border_names)}")
except KeyError:
    print("Unknown country, please use English or native name")

Languages: English, French
Borders: Central African Republic, Chad, Republic of the Congo, Equatorial Guinea, Gabon, Nigeria


## Pour en savoir plus

- Les API Rest: https://towardsdatascience.com/introduction-to-rest-apis-90b5d9676004
- La librairie [`requests`](https://requests.readthedocs.io/en/latest/)
- Développer une API en Python : https://openclassrooms.com/en/courses/4525361-realisez-un-dashboard-avec-tableau/5774811-creez-une-api-avec-flask